In [450]:
# import pip
# pip.main(['install', '--upgrade', 'snownlp'])
# pip.main(['install', '--upgrade', 'jieba'])
# pip.main(['install', '--upgrade', 'gensim'])
# pip.main(['install', '--upgrade', 'pyLDAvis'])
# pip.main(['install', '--upgrade', 'nltk'])

In [451]:
import pandas as pd
import numpy as np

from snownlp import SnowNLP

import jieba
import jieba.analyse
import jieba.posseg as pseg
import sys
import nltk

from collections import Counter

from nltk.corpus import stopwords
from nltk.corpus import treebank

from operator import itemgetter

from nltk.stem.snowball import SnowballStemmer

import re

from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

import os
import io
import json
import gzip
#import synonyms

from gensim import corpora, models, similarities
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans 
from sklearn.cluster import Birch 
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score 

import os 

import matplotlib.pyplot as plt
%matplotlib inline 

import matplotlib as mpl
 
from sklearn.manifold import MDS

import warnings
warnings.filterwarnings("ignore")
# nltk.download()

# <font size=5> Read data

<font size=4> Some preprocessing need be done...

In [452]:
jamQues = pd.read_csv(open('/Users/huangweipeng/Desktop/JAM Culture Transformation/Jam notebook&data&ppt/ctjam_question.csv','rU'))
jamQues.loc[jamQues['QuestionTitle'].map(lambda x: '巅峰' in str(x)), 'QuestionTitle'] = jamQues.loc[jamQues[
    'QuestionTitle'].map(lambda x: '巅峰' in str(x)), 'QuestionTitle'].map(lambda x: x.split('(')[0])
jamQues.loc[jamQues['QuestionTitle'] == 'Do you like our new Agile Open Office?', 'QuestionTitle'] = 'Do you like our new Agile Office?'
jamQues.loc[jamQues['QuestionTitle'] == '有谁用过WebSphere sMash， Project Zero ?', 'QuestionTitle'] = '有谁用过WebSphere sMash， ProjectZero?'
jamQues.loc[jamQues['QuestionTitle'].map(lambda x: '先客观地问自己，我对目前整体的团队氛围满意吗？' in x), 'QuestionTitle'] = '先客观地问自己，我对目前整体的团队氛围满意吗？'
jamQues.loc[jamQues['QuestionTitle'].map(lambda x: 'Angile' in x), 'QuestionTitle'] = jamQues.loc[jamQues['QuestionTitle'].map(lambda x: 'Angile' in x), 'QuestionTitle'].map(lambda x: x.replace(
            'Angile', 'Agile'))                                                                                                   
jamQues.loc[jamQues['QuestionTitle'].map(lambda x: 'openwisk' in x.lower()), 'QuestionTitle'] = jamQues.loc[jamQues['QuestionTitle'].map(lambda x: 'openwisk' in x), 'QuestionTitle'].map(lambda x: x.replace(
            'openwisk', 'openwhisk'))
jamQues['QuestionTitle'] = jamQues['QuestionTitle'].map(lambda x: str(x))
jamQues

,QuestionTitle,CreatedTime
0,Do you like our new Agile Office?,2017-08-17 14:31
1,CDL办公室管理的挑战：工作日有小朋友跟着家长来上班。一方面，学校放假期间小朋友不能自己呆在...,2017-08-30 09:42
2,工程師的日常生活裡，你覺得甚麼最需要去改變？(e.g. 工作態度，社交活動，生活習慣，創新思...,2017-08-16 13:45
3,"Dear IBMer, 你觉得你能够做些什么来让IBM重新在业界走上巅峰？",2017-08-21 19:21
4,除了项目以外， 你一周学习新技术的时间有多少？比如：一周1个小时，一周4个小时，一周10个小...,2017-08-14 18:01
5,IBM要转型成功，你认为是技术更重要，还是满足客户需求更重要？,2017-08-24 16:04
6,说到工程师文化，你想到的前三个词是什么？例如： 自由，效率，平等，尊重，信任，创新，简化流程...,2017-08-14 18:01
7,你最喜欢的Whitewater工具是什么？,2017-08-14 14:43
8,你希望工作环境发生的一个小改变是什么？,2017-08-14 18:01
9,你觉得IBM是技术导向的么？,2017-08-14 16:49


# <font size=5> Word Segmentation

<font size=4> Phase I:
1. Remove some stop words (The chinese stop words are downloaded from Internet, and I also add some stop words artificially)

2. Remove some words with unuseful part of speeach tag (artifical definition)


In [453]:
# read the txt file of chinese stopwords
def procTxtFile(filePath):
    fileContent = open(filePath)
    fileList = fileContent.read().replace(' ', '').replace("'", '').replace('\n', '').split(',')
    fileContent.close()
    return fileList

In [454]:
# stopwords for chinese and english
stopwords_chn = procTxtFile('/Users/huangweipeng/Desktop/JAM Culture Transformation/Jam notebook&data&ppt/stopwords_chinese.txt')
stopwords_chn += ['原因', '回答', '问题', '发生', '小事', '列出', '上做', '地问', '让你做', '时间', '用过', '让你在']
# import stopwords in English from NLKT
stopwords_eng = stopwords.words('english')
stopwords_eng += ['david', 'kenny', 'townhall', 'dear', '...', 'would', 'could', 'etc', 'ibmer', 'bue']
stopwords_total = set(stopwords_chn + stopwords_eng)
print('stopwords:'+str(len(stopwords_total)))
stopwords_total.difference_update({'', 'no', 'not'})
print('stopwords:'+str(len(stopwords_total)))
print(stopwords_total)

stopwords:903
stopwords:900
{'赖以', '极了', '怎么', '比', '或则', '总的说来', '并且', '开外', '嗬', '哪天', '反过来', '尔后', '曾', '尔', '尽管如此', '就是说', '孰知', '如此', '与否', '也好', 'hadn', '难道说', '般的', '至今', 'don', 'isn', '乘', '你们', 'its', '不单', '随时', 'what', '谁料', '譬如', '莫如', '如果', '呵呵', '之类', '不外乎', '还要', 'from', '再有', '呢', '对比', '儿', '借傥然', '前后', 'dear', '巴巴', '直到', 'haven', 'now', '除', 'down', 'had', '对待', '只消', 'shan', '替', '果然', '这样', 'about', '这么', '及其', '不如', 'townhall', '处在', '不论', '不管', 'ma', 'both', '以免', '虽然', '遵循', 'during', '不过', '嘿', '待', 'further', '第', '后', '顺着', '还有', '结果', 'an', '分别', '那', '所', '大', 'mightn', '截至', '即如', '为', '正值', 'theirs', '嘎', '她们', 'of', 'off', 'few', '那儿', '宁', '甚么', '只要', '以及', '嘛', '一来', '那么样', 'why', '可以', 'because', '由', '照', '一般', '不得', 'the', '阿', 'whom', '倘使', '的话', '漫说', '以至于', '一则', '像', '看', '让', '再其次', '出于', '慢说', '这个', 'm', 'doing', '哪怕', '上下', 'couldn', '上做', '嗳', '因了', '至于', 'itself', '换言之', '出来', '是', '但', '已', '着', 'hasn', '哎呀', '既然', '唉', '不尽', '而外', '那般', '

In [455]:
# save the stopwords as txt file
stopTotalFile = open('/Users/huangweipeng/Desktop/JAM Culture Transformation/Jam notebook&data&ppt/stopwords_total.txt', 'w+')
stopwords_string = '\n'.join(list(stopwords_total))
stopTotalFile.write(stopwords_string)

2880

In [456]:
# 调节单个词语的词频，使其更容易/不容易被划分出来
jieba.add_word('不限于', freq = 200, tag=None)
jieba.add_word('不需要', freq = 200, tag=None)
jieba.add_word('不能够', freq = 200, tag=None)
jieba.add_word('不可以', freq = 200, tag=None)
jieba.add_word('不想要', freq = 200, tag=None)
jieba.add_word('不希望', freq = 200, tag=None)
jieba.add_word('不理解', freq = 200, tag=None)
jieba.add_word('不了解', freq = 200, tag=None)
jieba.add_word('有没有', freq = 200, tag=None)
jieba.add_word('小确幸', freq = 200, tag=None)
jieba.add_word('充满热情', freq = 0, tag = None)
jieba.add_word('最想', freq = 200, tag = None)

In [457]:
# return the flag set of words, select the unuseful flags to be the stop_flag
def jiebaTextFlag(text):
    reg = re.compile(u"[\u4e00-\u9fa5]+")
# search for Chinese words, to use different package for English sentences and Chinese sentences
    flagList = []
    if reg.search(text):
        segList = pseg.lcut(text, HMM = True)
        for wordSeg in segList:
            try:
                [word, flag] = str(wordSeg).split('/')
                if (word.strip() not in stopwords_total) and (len(word.strip())>1):
                    flagList.append(flag)     
            except:
                continue
    else:
        segList = pos_tag(word_tokenize(text))
        for (word, flag) in segList:
            if (word.strip() not in stopwords_eng) and (len(word.strip())>1):
                flagList.append(flag)
                          
    return set(flagList)

In [458]:
jamQues['QuesText'] = jamQues['QuestionTitle'].map(lambda x: SnowNLP(x))
jamQues['QuesSimText'] = jamQues['QuesText'].map(lambda x: x.han)

In [459]:
flagSeries = jamQues['QuesSimText'].map(lambda x: jiebaTextFlag(x))
flagTotalSet = set()
for flagSet in flagSeries:
    flagTotalSet = flagTotalSet.union(flagSet)
print('All flags of the words: \n' +str(flagTotalSet))
print(flagSeries)

All flags of the words: 
{'VBG', 't', 'd', 'JJ', 'POS', 's', 'RB', 'a', 'l', 'vn', 'i', 'WRB', 'z', 'eng', 'm', 'c', 'NN', 'NNS', 'nr', 'x', 'vd', 'v', 'mq', 'DT', 'WP', 'NNP', 'VBP', 'f', 'TO', 'VBZ', 'n', 'JJS', 'VB', 'IN', 'r', 'ad', 'CD'}
0                     {VBP, IN, JJ, NNP}
1     {vn, i, n, z, eng, s, v, nr, f, a}
2                          {n, vn, i, v}
3                         {n, v, a, eng}
4                           {n, c, v, m}
5                         {n, v, a, eng}
6                        {d, n, m, v, a}
7                            {n, v, eng}
8                          {vn, n, v, m}
9                            {n, d, eng}
10                      {vn, a, n, v, l}
11                       {vn, n, m, eng}
12       {vn, t, d, n, eng, m, ad, v, a}
13                 {vn, n, eng, m, x, v}
14                                   {n}
15                             {VB, NNP}
16       {TO, DT, VB, NN, NNS, POS, NNP}
17                             {n, t, v}
18                 

In [460]:
# The flag of words to be dropped during word segementation
stopflags_chn = ['x', 'c', 'd', 'p', 't', 'f', 'r', 'm', 'mq', 'b', 'z']
stopflags_eng = ['CD', 'TO', 'WRB', 'DT', 'IN', 'WP', 'POS']

# return the word set of words segemented 
def jiebaClearText(text):
    reg = re.compile(u"[\u4e00-\u9fa5]+")
# search for Chinese words, in order to use different package for English sentences and Chinese sentences
    wordList = []
    if reg.search(text):
        segList = pseg.lcut(text, HMM = True)
        for wordSeg in segList:
            try:
                [word, flag] = str(wordSeg).split('/')
                if (word.strip() not in stopwords_total) and (len(word.strip())>1) and (flag not in stopflags_chn):
                    wordList.append(word)
            except:
                continue
    else:
        segList = pos_tag(word_tokenize(text))
        for (word, flag) in segList:
            if (word.strip() not in stopwords_eng) and (len(word.strip())>1) and (flag not in stopflags_eng):
                wordList.append(word)
# return the set of words for each questions, as I think the duplication of one word in each question is not necessary                          
    return tuple(set(wordList))

In [461]:
jamQues['QuesWords_1'] = jamQues['QuesSimText'].map(lambda x: jiebaClearText(x.lower()))
print(jamQues['QuesWords_1'])

0                            (like, agile, new, office)
1     (损坏, 工作日, 涂涂, 画画, 时有发生, cdl, 环境, 马克笔, 投影幕, 影响,...
2     (改变, 自由发挥, 思考, 日常生活, 工作, 创新, 态度, 热忱, 工程师, 生活习惯...
3                                 (ibm, 巅峰, 重新, 走上, 业界)
4                                      (技术, 学习, 项目, 小时)
5                     (技术, 需求, ibm, 满足, 成功, 转型, 客户, 重要)
6     (迭代, 流程, 解决问题, 信任, 效率, 想到, 自由, 创新, 工程师, 导向, 平等...
7                                  (工具, 喜欢, whitewater)
8                                      (工作, 希望, 环境, 改变)
9                                         (技术, 导向, ibm)
10    (技术, 销售, 经理, 测试, 深刻理解, 人才, 成功, 产品, 市场, 运维, 项目,...
11                                        (工作, 心情, lab)
12    (努力, 产业, 进行, ibm, 胜利, 帮助, cdl, 经历, 变革, 转型, 具体,...
13    (方面, 管理, 技能, 工作, 扩展, 沟通, 知识, transformation, 体...
14                                             (技术, 心目)
15                                          (love, ibm)
16      (ideas, autumn, bj, sep, building, enjoy, team)
17                                 (满意, 整体, 客观, 

<font size=4>Phase II:

Map the English words to Chinese words (The English words which have the same meaning with Chinese words cannot be put into directly the corpus used for clustering)

In [462]:
# return all words segmented by function jiebaClearText
def allWordSet(totalWord):
    wordsTotal = []
    for words in totalWord:
        wordsTotal.append(list(words))
    allWords = sum(wordsTotal, [])
    allSet = set(allWords)
    return allSet

In [463]:
allSet = allWordSet(pd.unique(jamQues['QuesWords_1']))

print(len(allSet))
print(allSet)
# Because some English words have the same meaning as the certain Chinese words,
# thus it's neccessary to do English-Chinese transformation before drop the words with low frequency

330
{'销售', '完成', '画画', 'git', '经历', '环境', '生活习惯', '阻碍', '不足', '建议', 'environment', '苦恼', '尊重', '应用', 'pls', 'technical', '认知', '抛开', 'fc', '办公区', '摔伤', '测试', '进行', 'approach', 'latest', '扩展', '开会', '士气', 'websphere', '上班', '画质', 'topic', '市场', '心目', 'hate', 'neither', 'love', '保持', '客户', '团队', 'part', '挑战', '重要', 'tool', '减少', '产业', '改进', 'worst', '技能', 'culture', '小朋友', '深刻理解', '概念', '转型', '成年人', '氛围', 'whitewater', '没有', '迭代', '忍受', '系统', 'hibernate', '次数', '放假', 'organization', '适合', 'ideas', '分配', 'autumn', '办公室', '巅峰', '最大', '类型', '留在', 'gvt', '社交活动', '小孩', '业界', '同事', '生命', '损坏', '改变', 'smash', 'first', '欠缺', 'cdl', '最好', '具体', '马克笔', '投影幕', 'recommend', '增加', '范畴', '红白机', '游戏', 'nan', 'working', '外界', '安全隐患', '方面', '上用', 'lab', '最多', '日常生活', '满足', '很高', '满意', '性质', '综上', 'invention', '相关', 'sep', 'spring', '代码', '单选框', '解决', '讨论', '文化', 'money', '职业规划', 'plan', 'jquery', 'test', '掌握', '家长', '人才', '变革', '创新', '不相干', '产品', 'transformation', '登录', '不能自己', 'enjoy', '期待', 'work', "'s

In [464]:
# read the mapping data and creat the mapping dict for English words to Chinese words
chnReg = re.compile(u"[\u4e00-\u9fa5]+")
engReg = re.compile('[a-z]+')

def creatMap(mapfile):
    mapcontext = open(mapfile).readlines()
    wordsMap = {}
    for ct in mapcontext:
        engWords = engReg.search(ct).group(0)
        chnWords = chnReg.search(ct).group(0)
        wordsMap[engWords] = chnWords
    return wordsMap

engChnMap = creatMap('/Users/huangweipeng/Desktop/JAM Culture Transformation/Jam notebook&data&ppt/engChnMap.txt')
print(engChnMap)
print(len(engChnMap))

{'activity': '活动', 'approach': '方法', 'autumn': '秋天', 'believe': '相信', 'challenging': '挑战', 'comfortable': '舒服', 'community': '社区', 'company': '公司', 'comprises': '组成', 'culture': '文化', 'development': '开发', 'disclosure': '公开', 'effective': '有效', 'enjoy': '享受', 'environment': '环境', 'fear': '害怕', 'greatest': '最大', 'hate': '讨厌', 'ideas': '想法', 'invention': '发明', 'keep': '继续', 'latest': '最近', 'learn': '学习', 'leverage': '利用', 'like': '喜欢', 'love': '喜欢', 'make': '使得', 'meeting': '会议', 'money': '金钱', 'months': '月', 'neither': '均不', 'new': '新的', 'open': '开源', 'organization': '公司', 'part': '部分', 'participate': '参加', 'past': '过去', 'person': '人', 'plan': '计划', 'pm': '项目经理', 'project': '项目', 'question': '问题', 'recommend': '推荐', 'secret': '秘密', 'security': '安全性', 'sep': '九月', 'sharing': '分享', 'skill': '技术', 'spend': '花费', 'stop': '停止', 'stronger': '强大', 'study': '学习', 'submit': '提交', 'team': '团队', 'technical': '技术', 'technology': '技术', 'test': '测试', 'thing': '事情', 'tool': '工具', 'topic': '主题', 'transf

In [465]:
def wordMap(words, mapdict, eng = True):
    wordList = []
    words = list(words)
    if eng:
        reg = re.compile('[a-zZ_A]+')
    else:
        reg = re.compile(u"[\u4e00-\u9fa5]+")
    for word in words:
        if reg.search(word):
            word = mapdict.get(word, word)
            wordList.append(word)
        else:
            wordList.append(word)
    return tuple(wordList)

In [466]:
# transform some English words to Chinese for dropping the low frequency words later
jamQues['QuesWords_2'] = jamQues['QuesWords_1'].map(lambda x: wordMap(x, engChnMap))
jamQues.head()

,QuestionTitle,CreatedTime,QuesText,QuesSimText,QuesWords_1,QuesWords_2
0,Do you like our new Agile Office?,2017-08-17 14:31,<snownlp.SnowNLP object at 0x135688cf8>,Do you like our new Agile Office?,"(like, agile, new, office)","(喜欢, agile, 新的, 办公室)"
1,CDL办公室管理的挑战：工作日有小朋友跟着家长来上班。一方面，学校放假期间小朋友不能自己呆在...,2017-08-30 09:42,<snownlp.SnowNLP object at 0x135688240>,CDL办公室管理的挑战：工作日有小朋友跟着家长来上班。一方面，学校放假期间小朋友不能自己呆在...,"(损坏, 工作日, 涂涂, 画画, 时有发生, cdl, 环境, 马克笔, 投影幕, 影响,...","(损坏, 工作日, 涂涂, 画画, 时有发生, cdl, 环境, 马克笔, 投影幕, 影响,..."
2,工程師的日常生活裡，你覺得甚麼最需要去改變？(e.g. 工作態度，社交活動，生活習慣，創新思...,2017-08-16 13:45,<snownlp.SnowNLP object at 0x13569a630>,工程师的日常生活里，你觉得甚么最需要去改变？(e.g. 工作态度，社交活动，生活习惯，创新思...,"(改变, 自由发挥, 思考, 日常生活, 工作, 创新, 态度, 热忱, 工程师, 生活习惯...","(改变, 自由发挥, 思考, 日常生活, 工作, 创新, 态度, 热忱, 工程师, 生活习惯..."
3,"Dear IBMer, 你觉得你能够做些什么来让IBM重新在业界走上巅峰？",2017-08-21 19:21,<snownlp.SnowNLP object at 0x13569a358>,"Dear IBMer, 你觉得你能够做些什么来让IBM重新在业界走上巅峰？","(ibm, 巅峰, 重新, 走上, 业界)","(ibm, 巅峰, 重新, 走上, 业界)"
4,除了项目以外， 你一周学习新技术的时间有多少？比如：一周1个小时，一周4个小时，一周10个小...,2017-08-14 18:01,<snownlp.SnowNLP object at 0x13531b048>,除了项目以外， 你一周学习新技术的时间有多少？比如：一周1个小时，一周4个小时，一周10个小...,"(技术, 学习, 项目, 小时)","(技术, 学习, 项目, 小时)"


<font size=4>Phase III:

Process the synonyms, map a word to another word with the same meaning (remove the duplication in the corpus)

In [467]:
# The new set of words after the transformation from English to Chinese
wordSet_1 = allWordSet(pd.unique(jamQues['QuesWords_2']))
# We need process the synonym in this set later
print('# All words: ' + str(len(wordSet_1)))
print(wordSet_1)

# All words: 303
{'销售', '完成', '画画', 'git', '经历', '环境', '分享', '生活习惯', '阻碍', '不足', '建议', '问题', '苦恼', '尊重', '应用', 'pls', '抛开', 'fc', '办公区', '摔伤', '测试', '进行', '扩展', '开会', '士气', 'websphere', '上班', '画质', '认知', '市场', '心目', '推荐', '保持', '客户', '团队', '挑战', '重要', '减少', '产业', '改进', '技能', '小朋友', '深刻理解', '会议', '概念', '社区', '转型', '成年人', '氛围', 'whitewater', '没有', '迭代', '忍受', '系统', 'hibernate', '次数', '放假', '事情', '适合', '分配', '办公室', '巅峰', '最大', '类型', '留在', 'gvt', '社交活动', '小孩', '业界', '同事', '有效', '生命', '主题', '损坏', '改变', 'smash', 'first', '欠缺', 'cdl', '最好', '具体', '马克笔', '投影幕', '增加', '红白机', '范畴', '游戏', 'nan', '外界', '公开', '安全隐患', '方面', '上用', 'lab', '最多', '日常生活', '满足', '很高', '满意', '性质', '综上', '相关', '计划', 'spring', '代码', '单选框', '解决', '讨论', '文化', '职业规划', 'jquery', '掌握', '家长', '人才', '变革', '创新', '不相干', '产品', '登录', '不能自己', '期待', "'s", '安全性', '努力', '享受', '文字', '发明', '厌烦', '想到', '自由', '情况', 'projectzero', '相信', 'sessions', '昂贵', '组成', 'bugzilla', '持续', '包括', '个人', '需求', '涂涂', 'ibm', '经理', '工作量', 'powerful', 'openwhisk'

In [468]:
# Load the vocabulary for synonyms
#def load_vocab(file_path):
#    '''
#    load vocab dict
#    '''   
#    fin = io.TextIOWrapper(
#        io.BufferedReader(
#            gzip.open(file_path)),
#        encoding='utf8',
#        errors='ignore')
#    vocab = json.loads(fin.read())  
#    return vocab
# build on load
#print(">> Synonyms on loading vocab ...")
#synVocab = load_vocab("data/words.nearby.json.gz")
#print('# Vocabulary for synonyms: ' + str(len(synVocab)))

In [469]:
# Creat another mapping dict for the words depend on the similarity detected by package synonyms
#def creatSynMap(wordlist):
#    Len = len(wordlist)
#    synMap = {}
#    synWords = set(synVocab.keys())
#    for i in range(Len-1):
#        for j in range(i+1, Len):
#            s1 = wordlist[i]
#            s2 = wordlist[j]
#            if (s1 in synWords) & (s2 in synWords):
#                sim = synonyms.compare(s1, s2)
#                if sim == 1.0:
#                    synMap[s1] = s2
#    return synMap

In [470]:
# Establish the mapping of synonyms
#reg = re.compile(u"[\u4e00-\u9fa5]+")
#wordList_1 = []
#for word in list(wordSet_1):
#    if reg.search(word):
#        wordList_1.append(word)
        
#synWordMap = creatSynMap(wordList_1)

As I have created the mapping of synonyms, thus the previous code is not need to run again. Actually, the package of synonyms has something not good, it's not friendly to use and its result is not good.

In [471]:
# The synWordMap need artifical filter
synWordMap = {
 '听听': '听到',
 '小朋友': '小孩',
 '开会': '会议',
 '看法': '态度',
 '办法': '方式',
 '欠缺': '缺少',
 '热忱': '热情',
 '种类': '类型',
 '留在': '逗留',
 '使用': '利用',
 '企业': '公司',
 '学': '学习',
 '上班': '工作',
 '软件': '技术',
  #'改变': '转型',
  #'转变': '转型'
}

# Some unuseful words to drop
dropSet = {'喜欢', '最大', '新的', '没有', '不能', '逗留', '停止', '继续', '听到', '缺少', '利用', '类型', '帮助', '希望'}

In [472]:
jamQues['QuesWords_3'] = jamQues['QuesWords_2'].map(lambda x: wordMap(x, synWordMap, eng = False))
jamQues.head()

,QuestionTitle,CreatedTime,QuesText,QuesSimText,QuesWords_1,QuesWords_2,QuesWords_3
0,Do you like our new Agile Office?,2017-08-17 14:31,<snownlp.SnowNLP object at 0x135688cf8>,Do you like our new Agile Office?,"(like, agile, new, office)","(喜欢, agile, 新的, 办公室)","(喜欢, agile, 新的, 办公室)"
1,CDL办公室管理的挑战：工作日有小朋友跟着家长来上班。一方面，学校放假期间小朋友不能自己呆在...,2017-08-30 09:42,<snownlp.SnowNLP object at 0x135688240>,CDL办公室管理的挑战：工作日有小朋友跟着家长来上班。一方面，学校放假期间小朋友不能自己呆在...,"(损坏, 工作日, 涂涂, 画画, 时有发生, cdl, 环境, 马克笔, 投影幕, 影响,...","(损坏, 工作日, 涂涂, 画画, 时有发生, cdl, 环境, 马克笔, 投影幕, 影响,...","(损坏, 工作日, 涂涂, 画画, 时有发生, cdl, 环境, 马克笔, 投影幕, 影响,..."
2,工程師的日常生活裡，你覺得甚麼最需要去改變？(e.g. 工作態度，社交活動，生活習慣，創新思...,2017-08-16 13:45,<snownlp.SnowNLP object at 0x13569a630>,工程师的日常生活里，你觉得甚么最需要去改变？(e.g. 工作态度，社交活动，生活习惯，创新思...,"(改变, 自由发挥, 思考, 日常生活, 工作, 创新, 态度, 热忱, 工程师, 生活习惯...","(改变, 自由发挥, 思考, 日常生活, 工作, 创新, 态度, 热忱, 工程师, 生活习惯...","(改变, 自由发挥, 思考, 日常生活, 工作, 创新, 态度, 热情, 工程师, 生活习惯..."
3,"Dear IBMer, 你觉得你能够做些什么来让IBM重新在业界走上巅峰？",2017-08-21 19:21,<snownlp.SnowNLP object at 0x13569a358>,"Dear IBMer, 你觉得你能够做些什么来让IBM重新在业界走上巅峰？","(ibm, 巅峰, 重新, 走上, 业界)","(ibm, 巅峰, 重新, 走上, 业界)","(ibm, 巅峰, 重新, 走上, 业界)"
4,除了项目以外， 你一周学习新技术的时间有多少？比如：一周1个小时，一周4个小时，一周10个小...,2017-08-14 18:01,<snownlp.SnowNLP object at 0x13531b048>,除了项目以外， 你一周学习新技术的时间有多少？比如：一周1个小时，一周4个小时，一周10个小...,"(技术, 学习, 项目, 小时)","(技术, 学习, 项目, 小时)","(技术, 学习, 项目, 小时)"


<font size=4>Phase IV:

Drop the low frequency words, as the dimension of these words is too sparse, actually it will not have effect on clustering.

In [473]:
# Drop the words which only exist in only one question
def dropLowFreq(totalWord):
    wordsTotal = []
    for words in totalWord:
        wordsTotal.append(list(words))
    allWords = sum(wordsTotal, [])
    allSet = set(allWords)
    onceWords = set(word for word in allSet if allWords.count(word) == 1).union(dropSet)
    textWords = [[word for word in words if word not in onceWords] for words in wordsTotal]
    wordSet = allSet.difference(onceWords)
    wordFrame = pd.DataFrame({'QuesWords_3': totalWord, 'QuesWords_4': textWords})
    #wordSet = wordSet.difference_update({'ibm', '喜欢'})
    return wordSet, wordFrame

In [474]:
wordSet, wordFrame = dropLowFreq(pd.unique(jamQues['QuesWords_3']))
print('# Remained words after English-Chinese and synonyms transformation: ' + str(len(wordSet)))

# Remained words after English-Chinese and synonyms transformation: 45


In [475]:
jamQues = jamQues.merge(wordFrame, on = 'QuesWords_3', how = 'left')
jamQues['QuesWords_4'] = jamQues['QuesWords_4'].map(lambda x: tuple(x))


In [476]:
jamQues[jamQues['QuesWords_4']==()].index

Int64Index([22, 23, 25, 29, 31, 51, 53, 54, 64, 69, 79, 84, 85, 86, 91], dtype='int64')

In [477]:
mylist

[22, 23, 25, 29, 31, 50, 52, 53, 63, 68, 76, 81, 82]

# <font size=5> LDA Topic Model

Actually, the LDA topic model can be skipped...

In [478]:
def ldaTopics(data, num, plt = False):
    # creat the dictionary, corpus nad the tf-idf weight matrix
    dictionary = corpora.Dictionary(data)
    corpus = [dictionary.doc2bow(text) for text in data]
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    # topic model lda
    lda = models.ldamodel.LdaModel(corpus = corpus_tfidf, id2word = dictionary, num_topics = num, iterations = 100)
    # CV Score to evaluate the results of LDA
    # ldaCm = CoherenceModel(model = lda, texts = textMatrix, dictionary = dictionary, coherence='c_v')
    # print (ldaCm.get_coherence())
    # LDA visualization
    if plt == True:
        pyLDAvis.enable_notebook()
        ldavis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
        pyLDAvis.show(ldavis)
    return corpus, lda

In [479]:
ldaWords = pd.unique(jamQues['QuesWords_4'])
quesCorpus, quesLda = ldaTopics(ldaWords, 12, plt = False)
# If plt=True, you will get a html with the visual topics 
# (but you need to interrupt the kernel to continue following cells) 
quesLda.show_topics(num_words = 5)    

[(7, '0.162*"团队" + 0.104*"开源" + 0.095*"改变" + 0.082*"公司" + 0.060*"技术"'),
 (5, '0.101*"公司" + 0.084*"开源" + 0.078*"转型" + 0.066*"重要" + 0.062*"项目"'),
 (4, '0.121*"学习" + 0.101*"项目" + 0.100*"api" + 0.098*"转变" + 0.077*"转型"'),
 (3, '0.187*"技术" + 0.103*"导向" + 0.078*"转型" + 0.074*"ibm" + 0.070*"family"'),
 (6, '0.109*"开发" + 0.098*"ibm" + 0.093*"文化" + 0.068*"成功" + 0.056*"改变"'),
 (0, '0.202*"agile" + 0.084*"团队" + 0.072*"项目" + 0.072*"办公室" + 0.068*"技术"'),
 (1, '0.118*"ibm" + 0.109*"挑战" + 0.104*"学习" + 0.078*"安全性" + 0.069*"方式"'),
 (11, '0.165*"工作" + 0.098*"工具" + 0.098*"知识" + 0.090*"热情" + 0.088*"产品"'),
 (10, '0.152*"公司" + 0.099*"工作" + 0.089*"会议" + 0.080*"环境" + 0.072*"建议"'),
 (2, '0.115*"影响" + 0.083*"工作" + 0.074*"态度" + 0.049*"热情" + 0.049*"工程师"')]

In [480]:
# Choose the topic with the largest propability as the final topic of a certain question
def topicOfQues(lda, corpus, num, data):
    topics = lda.get_document_topics(corpus, per_word_topics=True)
    all_topics = [(doc_topics, word_topics, word_phis) for doc_topics, word_topics, word_phis in topics]
    quesTopic = np.zeros(num)
    for i in range(num) :
        topicProb = all_topics[i][0]
        topicDict = {}
        for (j, prob) in topicProb:
            topicDict[prob] = j
        quesTopic[i] = topicDict[max(list(topicDict.keys()))]
    df = pd.DataFrame({'words': data, 'topic': quesTopic})  
    return df

In [481]:
wLen = len(ldaWords)
quesTopicFrame = topicOfQues(quesLda, quesCorpus, wLen, ldaWords)
quesTopicFrame = quesTopicFrame.rename(columns = {'words': 'QuesWords_4'})
quesTopicFrame = quesTopicFrame.merge(jamQues[['QuesSimText', 'QuesWords_4']].drop_duplicates(), on = 'QuesWords_4')

In [482]:
quesTopicCnt = quesTopicFrame.groupby('topic')['QuesSimText'].count().rename('questionCnt').reset_index()

# <font size = 5> K-Means Clustering

<font size=4>
1. word vector for each question
2. tf-idf weight matrix of all questions  

3. calculate the cosine distance between questions 
4. k-means clustering

In [483]:
# All the unique questions
quesList = pd.unique(jamQues['QuesSimText'])

In [484]:
# Words segementation depend on the filterred wordSet
# And also map the English words to Chinese words according to engChnMap
def wordForVec(text):
    textWord = []
    reg = re.compile('[a-zZ_A]+')
    segList = jieba.lcut(text.lower(), HMM = True)
    for wordSeg in segList:
        if reg.search(wordSeg):
            word = engChnMap.get(wordSeg, wordSeg)
        else:
            word = synWordMap.get(wordSeg, wordSeg)
        if word.strip() in wordSet:
            textWord.append(word)
    return list(set(textWord))

# Get the words vector for each question
fLen = len(wordSet)
qLen = len(quesList)
# whats difference when fLen is other values? fLen: columns' number qLen:rows' number
# wordMatrix = [[0]*fLen]*qLen
for j in range(qLen):
    ques = quesList[j]
    wordMatrix[j] = wordForVec(ques)
wordMatrix

[['agile', '办公室'],
 ['管理',
  'cdl',
  '办公室',
  '工作',
  '环境',
  '活动',
  '态度',
  '影响',
  '重要',
  '建议',
  '小孩',
  '同事',
  '挑战',
  '安全性'],
 ['改变', '工作', '创新', '热情', '态度', '工程师'],
 ['ibm'],
 ['技术', '学习', '项目'],
 ['技术', 'ibm', '成功', '转型', '重要'],
 ['导向', '工程师', '文化', '创新'],
 ['工具'],
 ['工作', '环境', '改变'],
 ['技术', '导向', 'ibm'],
 ['技术', '测试', '成功', '产品', '项目', '开发'],
 ['工作'],
 ['进行', 'ibm', 'cdl', '成功', '转型', '产品', '项目', '转变'],
 ['管理', '工作', '知识', '转型', '方式', '转变'],
 ['技术'],
 ['ibm'],
 ['团队'],
 ['团队'],
 ['ibm'],
 ['转型', '文化', 'api', '活动'],
 ['开源', '公司', '改变', '技术'],
 ['工作'],
 [],
 [],
 ['工作'],
 [],
 ['开源', '技术', '开发'],
 ['agile', '办公室'],
 ['技术', '工作'],
 [],
 ['技术', '开发'],
 [],
 ['工作'],
 ['公司', 'cdl', '管理层', '包括'],
 ['会议'],
 ['开源', '项目'],
 ['团队'],
 ['工作', '项目'],
 ['技术', '团队'],
 ['会议'],
 ['openwhisk'],
 ['工作'],
 ['api', '成功'],
 ['方式', '建议', '安全性'],
 ['公司'],
 ['产品', '技术'],
 ['工作', '热情'],
 ['ibm', '成功'],
 ['公司', '小孩', '工作'],
 ['工作', '挑战'],
 [],
 ['工作', '影响'],
 [],
 [],
 ['公司'],
 ['工作'],
 ['转型', '转变']

In [485]:
words = []
for word in wordMatrix:
    words.append(tuple(word))
print("~~~~~:")
wordsFrame = pd.DataFrame({'index': np.arange(qLen), 'words': words})
wordsFrame = wordsFrame.loc[wordsFrame['words'] != ()]
wordsFrame
print('# All questions: ' + str(len(quesList)))
print('# The questions without the feature: ' + str(len(quesList) - len(wordsFrame)))

~~~~~:
# All questions: 88
# The questions without the feature: 13


In [486]:
mylist = []
for i in range(88):
    if i not in wordsFrame.index:
        mylist.append(i)
mylist

[22, 23, 25, 29, 31, 50, 52, 53, 63, 68, 76, 81, 82]

In [487]:
quesFrame = pd.DataFrame({'index': np.arange(qLen), 'QuesSimText': quesList})
# whats the difference before vs after merge
wordsFrame = wordsFrame.merge(quesFrame, on = 'index')
wordsFrame = wordsFrame[['QuesSimText', 'words']].reset_index()
wordsFrame

,index,QuesSimText,words
0,0,Do you like our new Agile Office?,"(agile, 办公室)"
1,1,CDL办公室管理的挑战：工作日有小朋友跟着家长来上班。一方面，学校放假期间小朋友不能自己呆在...,"(管理, cdl, 办公室, 工作, 环境, 活动, 态度, 影响, 重要, 建议, 小孩,..."
2,2,工程师的日常生活里，你觉得甚么最需要去改变？(e.g. 工作态度，社交活动，生活习惯，创新思...,"(改变, 工作, 创新, 热情, 态度, 工程师)"
3,3,"Dear IBMer, 你觉得你能够做些什么来让IBM重新在业界走上巅峰？","(ibm,)"
4,4,除了项目以外， 你一周学习新技术的时间有多少？比如：一周1个小时，一周4个小时，一周10个小...,"(技术, 学习, 项目)"
5,5,IBM要转型成功，你认为是技术更重要，还是满足客户需求更重要？,"(技术, ibm, 成功, 转型, 重要)"
6,6,说到工程师文化，你想到的前三个词是什么？例如： 自由，效率，平等，尊重，信任，创新，简化流程...,"(导向, 工程师, 文化, 创新)"
7,7,你最喜欢的Whitewater工具是什么？,"(工具,)"
8,8,你希望工作环境发生的一个小改变是什么？,"(工作, 环境, 改变)"
9,9,你觉得IBM是技术导向的么？,"(技术, 导向, ibm)"


In [488]:
# All the questions need clustering
quesClusterSet = wordsFrame['QuesSimText']

In [489]:
def calcQuesDist(tokenizer, questions, pcaNum = False):
    # Creat the word frequency matrix
    vecMatrixTrans = CountVectorizer(tokenizer = tokenizer)
    vecMatrix = vecMatrixTrans.fit_transform(questions)
    featureList = vecMatrixTrans.get_feature_names()
    print(featureList)
    # get the tf-idf weighted matrix
    tfidfTrans = TfidfTransformer()
    tfidfMatrix = tfidfTrans.fit_transform(vecMatrix)
    vecWeight = tfidfMatrix.toarray()
    print('The dimension of word vector matrix: ' + str(len(vecWeight[0])))
    # calculate the cosine distance
    if pcaNum:
        pca = PCA(n_components = pcaNum)   
        vecWeight = pca.fit_transform(vecWeight)
        print('The dimension after PCA: ' + str(len(vecWeight[0])))
    quesDist = 1 - cosine_similarity(vecWeight)
    return vecWeight, quesDist

In [490]:
vecWeight, quesDist = calcQuesDist(wordForVec, quesClusterSet)

['agile', 'api', 'cdl', 'defect', 'family', 'ibm', 'openwhisk', '产品', '会议', '公司', '创新', '办公室', '包括', '同事', '团队', '学习', '安全性', '导向', '小孩', '工作', '工具', '工程师', '建议', '开发', '开源', '影响', '态度', '成功', '技术', '挑战', '改变', '文化', '方式', '活动', '测试', '热情', '环境', '知识', '管理', '管理层', '转变', '转型', '进行', '重要', '项目']
The dimension of word vector matrix: 45


In [491]:
#tf-idf weight matrix
vecWeight

array([[ 0.68623278,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.24842587, ...,  0.        ,
         0.28252751,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [492]:
# distance matrix between questions
quesDist

array([[  0.00000000e+00,   8.08463981e-01,   1.00000000e+00, ...,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00],
       [  8.08463981e-01,  -2.22044605e-16,   8.36985346e-01, ...,
          8.97677391e-01,   7.17472487e-01,   1.00000000e+00],
       [  1.00000000e+00,   8.36985346e-01,  -2.22044605e-16, ...,
          8.36753274e-01,   1.00000000e+00,   1.00000000e+00],
       ..., 
       [  1.00000000e+00,   8.97677391e-01,   8.36753274e-01, ...,
          0.00000000e+00,   1.00000000e+00,   1.00000000e+00],
       [  1.00000000e+00,   7.17472487e-01,   1.00000000e+00, ...,
          1.00000000e+00,   0.00000000e+00,   1.00000000e+00],
       [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00, ...,
          1.00000000e+00,   1.00000000e+00,   0.00000000e+00]])

In [493]:
def kmCluster(dist, num, vec):
    # K-Means Clustering
    num_clusters = num
    rng = np.random.RandomState(42)
    kmCluster = KMeans(n_clusters = num, precompute_distances = True, random_state = rng)
    print('KMeans Clustering: ')
    kmCluster.fit_transform(dist)
    clusters = kmCluster.labels_.tolist()
    print('# Questions in each cluster: ' + str(dict(Counter(clusters))))
    silhouette = silhouette_score(vec, clusters) # 平均轮廓系数  
    # sample_silhouette_values = silhouette_samples(X, y)  #(每个点的轮廓系数)  
    print('silhouette score: ' + str(silhouette)) 
    inertia = kmCluster.inertia_
    print('The sum of squared distances of samples to their closest cluster center: ' + str(inertia)) 
    return clusters

<font size=4> Without PCA

In [494]:
num_clusters = 14
clusters_km = kmCluster(quesDist, num_clusters, vecWeight)

KMeans Clustering: 
# Questions in each cluster: {0: 4, 5: 13, 9: 8, 7: 8, 3: 6, 1: 2, 11: 4, 2: 7, 10: 5, 13: 3, 8: 5, 12: 5, 6: 3, 4: 2}
silhouette score: 0.331000740652
The sum of squared distances of samples to their closest cluster center: 52.2393385169


<font size=4> With PCA

In [495]:
weightMatrix, pcaDist = calcQuesDist(wordForVec, quesClusterSet, 14)
num_clusters = 12
clusters_km_pca = kmCluster(pcaDist, num_clusters, weightMatrix)

['agile', 'api', 'cdl', 'defect', 'family', 'ibm', 'openwhisk', '产品', '会议', '公司', '创新', '办公室', '包括', '同事', '团队', '学习', '安全性', '导向', '小孩', '工作', '工具', '工程师', '建议', '开发', '开源', '影响', '态度', '成功', '技术', '挑战', '改变', '文化', '方式', '活动', '测试', '热情', '环境', '知识', '管理', '管理层', '转变', '转型', '进行', '重要', '项目']
The dimension of word vector matrix: 45
The dimension after PCA: 14
KMeans Clustering: 
# Questions in each cluster: {9: 4, 5: 6, 10: 6, 2: 9, 6: 6, 4: 6, 1: 13, 8: 5, 3: 5, 0: 5, 11: 3, 7: 7}
silhouette score: 0.397430973133
The sum of squared distances of samples to their closest cluster center: 98.3387433973


In [496]:
# Merge the questions, words and clusters information to one dataframe
# Also put the questions without clustering into one cluster
QuesKmCluster = pd.concat([pd.DataFrame({'cluster_nopca': clusters_km, 'cluster_pca': clusters_km_pca}), wordsFrame], axis = 1)
otherQues = list(set(quesList).difference(set(quesClusterSet)))
otherCluster = pd.DataFrame({'QuesSimText': otherQues, 'words': [()] * len(otherQues)})
otherCluster['cluster_nopca'] = 14
otherCluster['cluster_pca'] = 12
QuesKmCluster = pd.concat([QuesKmCluster, otherCluster], axis = 0)
QuesKmCluster

,QuesSimText,cluster_nopca,cluster_pca,index,words
0,Do you like our new Agile Office?,0,9,0.0,"(agile, 办公室)"
1,CDL办公室管理的挑战：工作日有小朋友跟着家长来上班。一方面，学校放假期间小朋友不能自己呆在...,5,5,1.0,"(管理, cdl, 办公室, 工作, 环境, 活动, 态度, 影响, 重要, 建议, 小孩,..."
2,工程师的日常生活里，你觉得甚么最需要去改变？(e.g. 工作态度，社交活动，生活习惯，创新思...,9,10,2.0,"(改变, 工作, 创新, 热情, 态度, 工程师)"
3,"Dear IBMer, 你觉得你能够做些什么来让IBM重新在业界走上巅峰？",7,2,3.0,"(ibm,)"
4,除了项目以外， 你一周学习新技术的时间有多少？比如：一周1个小时，一周4个小时，一周10个小...,3,6,4.0,"(技术, 学习, 项目)"
5,IBM要转型成功，你认为是技术更重要，还是满足客户需求更重要？,1,4,5.0,"(技术, ibm, 成功, 转型, 重要)"
6,说到工程师文化，你想到的前三个词是什么？例如： 自由，效率，平等，尊重，信任，创新，简化流程...,5,10,6.0,"(导向, 工程师, 文化, 创新)"
7,你最喜欢的Whitewater工具是什么？,5,5,7.0,"(工具,)"
8,你希望工作环境发生的一个小改变是什么？,9,1,8.0,"(工作, 环境, 改变)"
9,你觉得IBM是技术导向的么？,1,2,9.0,"(技术, 导向, ibm)"


In [497]:
QuesKmCluster.loc[QuesKmCluster[
    'QuesSimText'] == 'Do you like our new Agile Office?', 'QuesSimText'] = 'Do you like our new Agile Open Office?'
QuesKmCluster.loc[QuesKmCluster[
    'QuesSimText'] == '有谁用过WebSphere sMash， ProjectZero?', 'QuesSimText'] = '有谁用过WebSphere sMash， Project Zero ?'

In [498]:
wordsFrame

,index,QuesSimText,words
0,0,Do you like our new Agile Office?,"(agile, 办公室)"
1,1,CDL办公室管理的挑战：工作日有小朋友跟着家长来上班。一方面，学校放假期间小朋友不能自己呆在...,"(管理, cdl, 办公室, 工作, 环境, 活动, 态度, 影响, 重要, 建议, 小孩,..."
2,2,工程师的日常生活里，你觉得甚么最需要去改变？(e.g. 工作态度，社交活动，生活习惯，创新思...,"(改变, 工作, 创新, 热情, 态度, 工程师)"
3,3,"Dear IBMer, 你觉得你能够做些什么来让IBM重新在业界走上巅峰？","(ibm,)"
4,4,除了项目以外， 你一周学习新技术的时间有多少？比如：一周1个小时，一周4个小时，一周10个小...,"(技术, 学习, 项目)"
5,5,IBM要转型成功，你认为是技术更重要，还是满足客户需求更重要？,"(技术, ibm, 成功, 转型, 重要)"
6,6,说到工程师文化，你想到的前三个词是什么？例如： 自由，效率，平等，尊重，信任，创新，简化流程...,"(导向, 工程师, 文化, 创新)"
7,7,你最喜欢的Whitewater工具是什么？,"(工具,)"
8,8,你希望工作环境发生的一个小改变是什么？,"(工作, 环境, 改变)"
9,9,你觉得IBM是技术导向的么？,"(技术, 导向, ibm)"


<font size=4> The result with PCA is better, but some artifical processing is still need.

In [499]:
QuesKmCluster.loc[QuesKmCluster['QuesSimText'].map(lambda x: 'CDL办公室管理的挑战' in x), 'cluster_pca'] = 1  
QuesKmCluster.loc[QuesKmCluster['QuesSimText'] == '你参加过最大的项目，有多少人同时工作？', 'cluster_pca'] = 4  
#QuesKmCluster.loc[QuesKmCluster['QuesSimText'] == 'Is the working environment comfortable', 'cluster_pca'] = 12  
QuesKmCluster.loc[QuesKmCluster['QuesSimText'] == '你觉得开源软件会带给企业什么改变？', 'cluster_pca'] = 9 
QuesKmCluster.loc[QuesKmCluster['QuesSimText'].map(lambda x: ('api' in str(x).lower()) or ('jam' in str(x).lower()) or (
('openwhisk' in str(x).lower()))),'cluster_pca'] = 13 
QuesKmCluster.loc[QuesKmCluster['QuesSimText'].map(
    lambda x: '你觉得现在的工作量分配' in str(x).lower()),'cluster_pca'] = 0
QuesKmCluster.loc[QuesKmCluster['QuesSimText'].map(
    lambda x: '你认为你所在的产品或项目最需要什么人才' in x),'cluster_pca'] = 4
QuesKmCluster.loc[QuesKmCluster['QuesSimText'].map(
    lambda x: '你们的项目中开发用的是什么tool' in x),'cluster_pca'] = 4
QuesKmCluster.loc[QuesKmCluster['QuesSimText'].map(
    lambda x: '你觉得现在的工作量分配' in x),'cluster_pca'] = 0
QuesKmCluster.loc[QuesKmCluster['cluster_pca'] == 6, 'cluster_pca'] = 12
QuesKmCluster.loc[QuesKmCluster['QuesSimText'].map(lambda x: 'TVT' in x), 'cluster_pca'] = 1
QuesKmCluster.loc[QuesKmCluster['QuesSimText'].map(
    lambda x: '工程师' in x),'cluster_pca'] = 6
QuesKmCluster.loc[QuesKmCluster['QuesSimText'].map(lambda x: 'agile' in str(x).lower()), 'cluster_pca'] = 10 

In [500]:
# Name the topics
QuesKmCluster['topic'] = np.nan
QuesKmCluster.loc[QuesKmCluster['cluster_pca'] == 0, 'topic'] = 'work'
QuesKmCluster.loc[QuesKmCluster['cluster_pca'] == 1, 'topic'] = 'company manage'
QuesKmCluster.loc[QuesKmCluster['cluster_pca'] == 2, 'topic'] = 'technique'
QuesKmCluster.loc[QuesKmCluster['cluster_pca'] == 3, 'topic'] = 'IBM'
QuesKmCluster.loc[QuesKmCluster['cluster_pca'] == 4, 'topic'] = 'project'
QuesKmCluster.loc[QuesKmCluster['cluster_pca'] == 5, 'topic'] = 'team'
QuesKmCluster.loc[QuesKmCluster['cluster_pca'] == 6, 'topic'] = 'engineer'
QuesKmCluster.loc[QuesKmCluster['cluster_pca'] == 7, 'topic'] = 'work+technique'
QuesKmCluster.loc[QuesKmCluster['cluster_pca'] == 8, 'topic'] = 'learn+technique'
QuesKmCluster.loc[QuesKmCluster['cluster_pca'] == 9, 'topic'] = 'open source'
QuesKmCluster.loc[QuesKmCluster['cluster_pca'] == 10, 'topic'] = 'environment+agile'
QuesKmCluster.loc[QuesKmCluster['cluster_pca'] == 11, 'topic'] = 'transformation'
QuesKmCluster.loc[QuesKmCluster['cluster_pca'] == 12, 'topic'] = 'others'
QuesKmCluster.loc[QuesKmCluster['cluster_pca'] == 13, 'topic'] = 'jam/api/openwhisk'
QuesKmCluster = QuesKmCluster.rename(columns = {'cluster_pca': 'cluster'})      

In [505]:
# Something worng need aritifical corrected, I don't know why~
QuesKmCluster = QuesKmCluster[['QuesSimText', 'cluster', 'words', 'topic']]
QuesKmCluster.loc[64, 'cluster'] = 12
QuesKmCluster

,QuesSimText,cluster,words,topic
0,Do you like our new Agile Open Office?,10,"(agile, 办公室)",environment+agile
1,CDL办公室管理的挑战：工作日有小朋友跟着家长来上班。一方面，学校放假期间小朋友不能自己呆在...,1,"(管理, cdl, 办公室, 工作, 环境, 活动, 态度, 影响, 重要, 建议, 小孩,...",company manage
2,工程师的日常生活里，你觉得甚么最需要去改变？(e.g. 工作态度，社交活动，生活习惯，创新思...,6,"(改变, 工作, 创新, 热情, 态度, 工程师)",engineer
3,"Dear IBMer, 你觉得你能够做些什么来让IBM重新在业界走上巅峰？",2,"(ibm,)",technique
4,除了项目以外， 你一周学习新技术的时间有多少？比如：一周1个小时，一周4个小时，一周10个小...,12,"(技术, 学习, 项目)",others
5,IBM要转型成功，你认为是技术更重要，还是满足客户需求更重要？,4,"(技术, ibm, 成功, 转型, 重要)",project
6,说到工程师文化，你想到的前三个词是什么？例如： 自由，效率，平等，尊重，信任，创新，简化流程...,6,"(导向, 工程师, 文化, 创新)",engineer
7,你最喜欢的Whitewater工具是什么？,5,"(工具,)",team
8,你希望工作环境发生的一个小改变是什么？,1,"(工作, 环境, 改变)",company manage
9,你觉得IBM是技术导向的么？,2,"(技术, 导向, ibm)",technique


In [506]:
QuesKmCluster.to_csv('QuesCluster_2.csv', index = False)

# <font size=4> It seems that keywords extraction has nothing to do...
I try to do question clustering based on the keywords of questions, but it seemed not work ~

In [503]:
def ExtractKeywords(text, num = 10, weight = False, pos = (), method = 'tfidf', span = 10):
    jieba.analyse.set_stop_words('stopwords_total.txt')
    if method == 'tfidf':
        keywords = jieba.analyse.extract_tags(text, topK = num, withWeight = weight, allowPOS = pos)
    else:
        textRank = jieba.analyse.TextRank()
        textRank.span = span
        keywords = jieba.analyse.textrank(text, topK = num,  withWeight = weight, allowPOS = pos)
    return tuple(keywords)

In [504]:
ExtractKeywords('今年最想学的知识是？')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa8 in position 5580: invalid start byte

In [ ]:
jamQues['QuesKeyText'] = jamQues['QuesSimText']
jamQues.loc[jamQues['QuesKeyText'].map(lambda x: '？' in x), 'QuesKeyText'] = jamQues.loc[
    jamQues['QuesKeyText'].map(lambda x: '？' in x), 'QuesSimText'].map(lambda x: x. split('？')[0])

In [ ]:
quesTag = ('a', 'ad', 'eng', 'i', 'l', 'n', 'nr', 'nrt', 's', 'v', 'vd', 'vn')
jamQues['QuesKeywords'] = jamQues['QuesKeyText'].map(lambda x: ExtractKeywords(x.lower(), pos = quesTag, method = 'tfidf'))

In [ ]:
def procKeyWords(data):
    regEng = re.compile('[a-zZ_A]+')
    regChn = re.compile(u"[\u4e00-\u9fa5]+")
    keyList = []
    for d in list(data):
        if regEng.search(d):
            word = engChnMap.get(d, d)
        elif regChn.search(d):
            word = synWordMap.get(d, d)
        else:
            word = d
        if word in wordSet:
            keyList.append(word)
    return tuple(keyList)

In [ ]:
jamQues['QuesUsedKey'] = jamQues['QuesKeywords'].map(lambda x: procKeyWords(x))

In [ ]:
def keyForVec(text):
    keywords = ExtractKeywords(text.lower(), pos = quesTag, method = 'tfidf')
    keys = procKeyWords(keywords)
    if len(keys) == 0:
        keys = wordForVec(text)
    return keys

In [ ]:
quesKeysFrame = jamQues[['QuesSimText', 'QuesKeyText', 'QuesUsedKey']].drop_duplicates()
keysFrame = quesKeysFrame.loc[quesKeysFrame['QuesUsedKey'] != ()].reset_index()
print('# All questions: ' + str(len(quesKeysFrame)))
print('# The questions without the feature: ' + str(len(quesKeysFrame) - len(keysFrame)))

In [ ]:
keysVec, keysDist = calcQuesDist(keyForVec, keysFrame['QuesKeyText'], 9)

In [ ]:
num_clusters = 11
clusters_km_keys = kmCluster(keysDist, num_clusters, keysVec)

In [ ]:
QuesKeysCluster = pd.concat([pd.DataFrame({'cluster_keywords': clusters_km_keys}), keysFrame], axis = 1)

In [ ]:
otherQues = list(set(quesList).difference(set(keysFrame['QuesSimText'])))
otherCluster = pd.DataFrame({'QuesSimText': otherQues})
otherCluster['cluster_keywords'] = num_clusters
otherCluster = otherCluster.merge(quesKeysFrame, on = 'QuesSimText', how = 'left')
QuesKeysCluster = pd.concat([QuesKeysCluster, otherCluster], axis = 0)
len(QuesKeysCluster)